### 第一步:我們先import我們所需的package

In [225]:
import torch
import torch.nn.functional as F
import random

### 第二步:建立初始值
我們此範例使用3D padding 去驗證我們的公式。

In [229]:
input_size = (1, 3, 4, 12, 12)
kernel_size = (3,3, 3, 3, 3)
Input = torch.randint(low=0, high=255, size=input_size)
# 根據only integer inference,我們的input的zero point會有值
Input_zero_point = random.randint(1, 255)
# (out_channels, in_channels, kernel_height, kernel_width)
weight = torch.randint(low=0, high=255, size=kernel_size)
print(Input)

print(Input - Input_zero_point)
x = F.pad(Input - Input_zero_point,(1,1,1, 1, 1, 1),value=0)
print(x.shape)
base_line_output = F.conv3d(x, weight, stride=1)
print(base_line_output.shape)

tensor([[[[[137, 234, 219,  ...,  63, 130,  46],
           [ 46, 187,  81,  ..., 240, 228,  52],
           [ 80, 197, 177,  ...,  75,  61,   0],
           ...,
           [ 65,  98,  34,  ...,  36,  51, 220],
           [ 71,  95, 128,  ..., 222, 147, 141],
           [231, 151, 250,  ..., 241, 222, 190]],

          [[217, 253, 214,  ...,   5,  72, 136],
           [ 26, 227, 112,  ..., 199, 144, 199],
           [ 57,  31, 184,  ...,  16,  46, 214],
           ...,
           [123,  27, 149,  ..., 143,  28, 220],
           [247, 191,  48,  ..., 247,   0, 224],
           [  5,   9,  16,  ...,  96, 130,  90]],

          [[ 21, 104,  81,  ...,  13, 198, 154],
           [182, 129, 135,  ...,  29, 196, 231],
           [ 15, 113,  97,  ..., 112,  62, 122],
           ...,
           [131, 139,  69,  ..., 143,  68,  91],
           [ 38,  21, 249,  ...,  33, 240, 190],
           [ 82,  39, 147,  ..., 232,  46, 216]],

          [[ 90, 179, 117,  ..., 130, 182, 102],
           [174

現在我們有BaseLine了
而BaseLine Formula為
$y = \sum{(x_q-x_{zp})(w_q-w_{zp})}$
我們的zp在做training的時候就為0了，所以為
$y = \sum{(x_q-x_{zp})(w_q)}$

接者我們就要驗證我們的算法是否可以實現
$y = \sum{x_qw_q} - x_{zp}\sum{w_q}$


我們原本的算法如果layer有做Padding會導致兩者出來的結果不同

In [241]:
res = F.conv3d(Input, weight, stride=1, padding=1)
print(res.shape)
print(base_line_output.shape)
res = res - (Input_zero_point * weight.sum(dim=(1, 2, 3, 4))).view(1, -1, 1, 1, 1)
loss = F.mse_loss(res.float(), base_line_output.float())
print(f"與Baseline的loss為{loss}!")

torch.Size([1, 3, 4, 12, 12])
torch.Size([1, 3, 4, 12, 12])
與Baseline的loss為284103737344.0!


會發現都無法解決，因此我們提供了一個方法

In [ ]:
custom_padding_input = F.pad(Input,pad=(1, 1, 1, 1, 1, 1),value=Input_zero_point)
output = F.conv3d(custom_padding_input, weight, stride=1)
output = output - (Input_zero_point * weight.sum(dim=(1, 2, 3, 4))).view(-1, 1, 1, 1)
check = F.mse_loss(output.float(), base_line_output.float())
print(f"與Baseline的loss為{check}!")

tensor([1560624, 1778712, 1504464])
tensor([[[[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],

          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],

          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],

          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],


  